In [1]:
import asyncio
import io
import glob
import os
import sys
import time
import uuid
import requests
import numpy as np
import pandas as pd
from urllib.parse import urlparse
from io import BytesIO
# To install this module, run:
# python -m pip install Pillow
from PIL import Image, ImageDraw
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.vision.face.models import TrainingStatusType, Person
from azure.cognitiveservices.vision.face.models import FaceAttributeType

ModuleNotFoundError: No module named 'azure'

In [2]:
#import vision API 
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

from array import array

In [3]:
#import packages for OpenCV
%matplotlib inline

from pylab import *
import cv2 as cv

In [4]:
#Retrive from Face Resource - Resource Management
faceKEY = "1ab3848ea22649299da961e8feb813a4"
faceENDPOINT = "https://face-rec-app.cognitiveservices.azure.com/"

In [5]:
# Create an authenticated FaceClient.
face_client = FaceClient(faceENDPOINT, CognitiveServicesCredentials(faceKEY))

In [7]:
#Create an authenticated Computer Vision Client
computervision_client = ComputerVisionClient(compvision_endpoint, CognitiveServicesCredentials(compvision_subscription_key))

In [8]:
# Attributes you want returned with the API call, a list of FaceAttributeType enum (string format)
face_attributes = ['headPose', 'glasses', 'occlusion', 'blur', 'exposure', 'noise']

In [9]:
def faceApp(image):
    listofIssues = []
    
    #possible issues
    
    rolled = 'head is tilted to the side'
    yawed = 'face is turned to the side'
    pitched = 'head is foreward or backward titled'
    tiltCentred = 'head and face is correctly positioned'
    rolledYawed = 'head is tilted to the side and face is turned to the side'
    rolledPitched = 'head is tilted to the side and head is foreward or backward tilted'
    pitchedYawed = 'head is foreward or backward tilted and face is turned to the side'
    rolledYawedPitched = 'head is tilted to the side, face is turned to the side, and head is forward or backward tilted'
    
    glasses = 'Face has glasses'
    noGlasses = 'Face does not have glasses'
    
    glarePresent = 'There is glare'
    noGlare = 'There is no glare'
    
    Light = 'The image has too much light' 
    faceLight = 'The face is too brightly lit (possibly due to flash)'
    Dark = 'The image is too dark'
    faceDark = 'The face is poorly lit'
    correctLighting = 'Image is correctly lit'    
    
    #loading the image
    local_image = open(image, "rb")
    image_size = Image.open(image).size
    image_width, image_height = image_size
    
    # Detect a face with attributes, returns a list[DetectedFace]
    detected_faces = face_client.face.detect_with_stream(local_image, return_face_attributes=face_attributes)
    faceno = 0
    for face in detected_faces:
        faceno += 1
    if not detected_faces:
        listofIssues = ['No face detected', 'No face detected', 'No face detected', 'No face detected', 'No face detected', 'No face detected', 'No face detected', 'No face detected']
        return listofIssues
    elif faceno >1:
        listofIssues = ['Multiple faces', 'Multiple faces', 'Multiple faces', 'Multiple faces', 'Multiple faces', 'Multiple faces', 'Multiple faces', 'Multiple faces']
        return listofIssues
    else:
        head_pose_string = str(face.face_attributes.head_pose)
        a, r, y, p = head_pose_string.split(',')
        roll = r[8:]
        roll_num = abs(float(roll))
        yaw = y[7:]
        yaw_num = abs(float(yaw))
        pitch = p[9:-1]
        pitch_num = abs(float(pitch))
    
        if (roll_num > 15) and (yaw_num > 15) and (pitch_num > 15):
            listofIssues.append(rolledYawedPitched)
        elif (roll_num > 15) and (yaw_num > 15):
            listofIssues.append(rolledYawed)
        elif (roll_num > 15) and (pitch_num > 15):
            listofIssues.append(rolledPitched)
        elif (yaw_num > 15) and (pitch_num > 15):
            listofIssues.append(pitchedYawed)
        elif roll_num > 15:
            listofIssues.append(rolled)
        elif yaw_num > 15:
            listofIssues.append(yawed)
        elif pitch_num > 15:
            listofIssues.append(pitched)
        else:
            listofIssues.append(tiltCentred)
        
        glasses_string = str(face.face_attributes.glasses)
        if glasses_string.__contains__("no_glasses") == True:
            listofIssues.append(noGlasses)
        else:
            listofIssues.append(glasses)
        
        #glare
    
        image_in = cv.cvtColor(cv.imread(image), cv.COLOR_BGR2RGB); 
        h, s, v = cv.split(cv.cvtColor(image_in, cv.COLOR_RGB2HSV))
    
        # Find all pixels that are not very saturated
        nonSat = s < 180 #this constant is from sample code

        # Slightly decrease the area of the non-satuared pixels by a erosion operation.
        disk = cv.getStructuringElement(cv.MORPH_ELLIPSE,(3,3))
        nonSat = cv.erode(nonSat.astype(np.uint8), disk)

        # Set all brightness values, where the pixels are still saturated to 0.
        v2 = v.copy()
        v2[nonSat == 0] = 0;
    
        glare = v2>200 #this constant is from sample code
    
        countTrue = np.count_nonzero(glare == True)
        countFalse = np.count_nonzero(glare == False)
        glarepercent = (countTrue/(countTrue+countFalse))*100
        
        #this glare has to be such a high percent due to the background
    
        if glarepercent > 10: 
            listofIssues.append(glarePresent)
        else:
            listofIssues.append(noGlare)
    
        #too dark or light photo
    
        #compared to face and background
        if (v.mean() - faceV.mean()) > 50:
            listofIssues.append(faceDark)
        elif (v.mean() - faceV.mean()) < -50:
            listofIssues.append(faceLight)
        #overall photo light and dark
        elif v.mean() > 200:
            listofIssues.append(Light)
        elif v.mean() < 50:
            listofIssues.append(Dark)
        else:
            listofIssues.append(correctLighting)
 
    
        return listofIssues

<ipython-input-9-bc637cafc6b5>:180: RuntimeWarning: invalid value encountered in true_divide
  contrast = (max-min)/(max+min)


,image,HP,HR,Glass,EV,FV,Glared,S,L
0,C:\Users\nidhimh\Documents\Face\faces\00000000...,No face detected,No face detected,No face detected,No face detected,No face detected,No face detected,No face detected,No face detected
1,C:\Users\nidhimh\Documents\Face\faces\00000000...,Multiple faces,Multiple faces,Multiple faces,Multiple faces,Multiple faces,Multiple faces,Multiple faces,Multiple faces
2,C:\Users\nidhimh\Documents\Face\faces\01.jpg,Face is horizontally and vertically centred,head and face is correctly positioned,Face does not have glasses,Eyes are not occluded,Forehead is not occluded,There is no glare,There are shadows,Image is correctly lit
3,C:\Users\nidhimh\Documents\Face\faces\01a.jpg,Face is not vertically centred,head is tilted to the side,Face has glasses,Eyes are not occluded,Forehead is not occluded,There is no glare,There are shadows,The face is too brightly lit (possibly due to ...
4,C:\Users\nidhimh\Documents\Face\faces\02.jpg,Face is horizontally and vertically centred,head and face is correctly positioned,Face has glasses,Eyes are not occluded,Forehead is not occluded,There is glare,There are no shadows,Image is correctly lit
